In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
housing = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')
housing.head()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.isna().sum()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
import matplotlib.pyplot as plt

hist = housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
housing[housing['housing_median_age'] > 50]['housing_median_age'].value_counts()

In [ ]:
housing.drop(housing[housing['housing_median_age'] > 51].index, inplace=True)

In [ ]:
housing[housing['median_house_value'] > 450000]['median_house_value'].value_counts()

In [ ]:
housing.drop(housing[housing['median_house_value'] > 500000].index, inplace=True)

In [ ]:
housing.isna().sum()

In [ ]:
housing['total_bedrooms'].value_counts()

In [ ]:
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(housing['total_bedrooms'].mean())
housing = housing.reset_index(drop=True)

housing.isna().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

housing = pd.get_dummies(housing, columns=['ocean_proximity'], drop_first=True)
housing

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
housing['median_income_category'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
# housing['median_income_category'].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['median_income_category']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]


In [ ]:
strat_train_set.drop('median_income_category', axis=1, inplace=True)
strat_test_set.drop('median_income_category', axis=1, inplace=True)

In [ ]:
X_train = strat_train_set.drop('median_house_value', axis=1)
y_train = strat_train_set['median_house_value']

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value']

X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
# !pip install lazypredict

# from lazypredict.Supervised import LazyRegressor
# from sklearn.model_selection import train_test_split


# # Initialize LazyRegressor
# regressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# # Train and evaluate models
# models, predictions = regressor.fit(X_train, X_test, y_train, y_test)

# # Display results
# print(models)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

# X_train ve X_test'inizi ölçeklendirin
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest modelini oluşturun
model = RandomForestRegressor(random_state=42)

# Çapraz doğrulama ile tahmin skorunu hesaplayın
scores = cross_val_score(model, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=3)
mse_scores = -scores  # Negatif MSE'yi pozitif yapmak için
rmse_scores = np.sqrt(mse_scores)

# Sonuçları yazdırın
print("MSE Skorları:", mse_scores)
print("RMSE Skorları:", rmse_scores)
print("Ortalama RMSE Skoru:", rmse_scores.mean())